# �� Análisis de Calidad de Datos (DQA)

## Justificación Académica

> **Justificación:** Según García et al. (2016) en *"Big Data Preprocessing"*, la auditoría de valores atípicos y nulos es crítica antes de cualquier modelado. En Tracer Studies, validar la consistencia lógica (Edad vs Año Graduación) es vital para evitar sesgos (Lawless, 2003).

**Objetivo**: Realizar un análisis exploratorio de calidad de datos (EDA/DQA) para identificar:
1. Valores nulos críticos
2. Edades imposibles
3. Inconsistencias lógicas
4. Registros a eliminar

**Datos de Entrada**: `../Encuesta recien graduados - pregrado(1).xlsx`  
**Salida**: `./data/processed/clean_survey_data.csv`

---

In [1]:
# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
import pandas as pd
import numpy as np
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Paths (datos crudos en raíz, procesados en v2)
DATA_RAW = Path("../Encuesta recien graduados - pregrado(1).xlsx")
DATA_OUTPUT = Path("data/processed/clean_survey_data.csv")

# Crear directorio si no existe
DATA_OUTPUT.parent.mkdir(parents=True, exist_ok=True)

print("✅ Configuración cargada")
print(f"   Archivo entrada: {DATA_RAW}")
print(f"   Archivo salida: {DATA_OUTPUT}")

✅ Configuración cargada
   Archivo entrada: ../Encuesta recien graduados - pregrado(1).xlsx
   Archivo salida: data/processed/clean_survey_data.csv


---
## 1. Carga de Datos Crudos

In [2]:
# Cargar Excel
df = pd.read_excel(DATA_RAW)

print(f"📊 DATOS CARGADOS")
print(f"   Registros: {len(df)}")
print(f"   Columnas: {len(df.columns)}")
print(f"\n🔍 Vista previa de columnas:")
for i, col in enumerate(df.columns[:15]):
    print(f"   [{i}] {col[:60]}...")

📊 DATOS CARGADOS
   Registros: 380
   Columnas: 42

🔍 Vista previa de columnas:
   [0] 1.1 Edad:...
   [1] 1.2 Género:...
   [2] 3. Evaluación de Habilidades Blandas...
   [3] A continuación, por favor evalúa qué tan bien consideras que...
   [4] A continuación, por favor evalúa qué tan bien consideras que...
   [5] A continuación, por favor evalúa qué tan bien consideras que...
   [6] A continuación, por favor evalúa qué tan bien consideras que...
   [7] A continuación, por favor evalúa qué tan bien consideras que...
   [8] A continuación, por favor evalúa qué tan bien consideras que...
   [9] A continuación, por favor evalúa qué tan bien consideras que...
   [10] 4. Inserción Laboral...
   [11] 4.1 ¿Actualmente estás trabajando?...
   [12] 4.1.2 País:...
   [13] 4.1.3 ¿A qué tipo pertenece la institución en la que laboras...
   [14] 4.1.4 Modalidad de trabajo:...


---
## 2. Análisis de Valores Nulos

In [3]:
# Columnas críticas para el análisis
CRITICAL_COLS = {
    11: "trabaja_actualmente",
    16: "correspondencia_formacion",
    35: "asignaturas_relevantes",  # VITAL para vectores técnicos
    40: "carrera"
}

print("=" * 60)
print("🔍 ANÁLISIS DE NULOS EN COLUMNAS CRÍTICAS")
print("=" * 60)

for idx, name in CRITICAL_COLS.items():
    col = df.iloc[:, idx]
    n_null = col.isna().sum()
    pct_null = 100 * n_null / len(df)
    status = "🔴 CRÍTICO" if pct_null > 10 else "🟡 MODERADO" if pct_null > 5 else "🟢 OK"
    print(f"\n[{idx}] {name}:")
    print(f"    Nulos: {n_null} ({pct_null:.1f}%) {status}")

# Análisis especial de Asignaturas (columna 35)
print("\n" + "=" * 60)
print("📝 ANÁLISIS DETALLADO: Asignaturas Relevantes (col 35)")
print("=" * 60)
asig_col = df.iloc[:, 35]
print(f"Nulos: {asig_col.isna().sum()}")
print(f"Vacíos/Blancos: {(asig_col.astype(str).str.strip() == '').sum()}")
print(f"Registros válidos: {(~asig_col.isna() & (asig_col.astype(str).str.strip() != '')).sum()}")

🔍 ANÁLISIS DE NULOS EN COLUMNAS CRÍTICAS

[11] trabaja_actualmente:
    Nulos: 0 (0.0%) 🟢 OK

[16] correspondencia_formacion:
    Nulos: 171 (45.0%) 🔴 CRÍTICO

[35] asignaturas_relevantes:
    Nulos: 1 (0.3%) 🟢 OK

[40] carrera:
    Nulos: 0 (0.0%) 🟢 OK

📝 ANÁLISIS DETALLADO: Asignaturas Relevantes (col 35)
Nulos: 1
Vacíos/Blancos: 0
Registros válidos: 379


---
## 3. Detección de Edades Imposibles

In [4]:
# Columna de edad (índice 0)
edad_col = pd.to_numeric(df.iloc[:, 0], errors='coerce')

print("=" * 60)
print("👤 ANÁLISIS DE EDADES")
print("=" * 60)

print(f"\nEstadísticas básicas:")
print(edad_col.describe().round(1))

# Detección de anomalías
edad_baja = edad_col < 20
edad_alta = edad_col > 60
edad_nula = edad_col.isna()

print(f"\n🚨 ANOMALÍAS DETECTADAS:")
print(f"   Edad < 20: {edad_baja.sum()} registros")
print(f"   Edad > 60: {edad_alta.sum()} registros")
print(f"   Edad nula: {edad_nula.sum()} registros")

# Marcar filas problemáticas
df['_flag_edad_invalida'] = edad_baja | edad_alta

if (edad_baja | edad_alta).sum() > 0:
    print(f"\n📋 Registros con edad problemática:")
    problemas_edad = df[edad_baja | edad_alta].iloc[:, [0, 40]]
    print(problemas_edad.head(10))

👤 ANÁLISIS DE EDADES

Estadísticas básicas:
count    380.0
mean      25.1
std        2.8
min       21.0
25%       23.0
50%       24.0
75%       26.0
max       40.0
Name: 1.1 Edad:, dtype: float64

🚨 ANOMALÍAS DETECTADAS:
   Edad < 20: 0 registros
   Edad > 60: 0 registros
   Edad nula: 0 registros


---
## 4. Validación de Consistencia Lógica

In [5]:
# Verificar consistencia: trabaja + correspondencia
trabaja = df.iloc[:, 11] == 'Si'
correspondencia = pd.to_numeric(df.iloc[:, 16], errors='coerce')

# Inconsistencia: Dice que NO trabaja pero tiene correspondencia laboral
inconsistente = (~trabaja) & (correspondencia > 0)

print("=" * 60)
print("⚠️ ANÁLISIS DE CONSISTENCIA LÓGICA")
print("=" * 60)

print(f"\n1. ¿No trabaja pero tiene correspondencia laboral?")
print(f"   Registros inconsistentes: {inconsistente.sum()}")

# Verificar año de graduación vs encuesta
anio_grad = pd.to_numeric(df.iloc[:, 41], errors='coerce')
print(f"\n2. Años de graduación:")
print(anio_grad.value_counts().sort_index())

df['_flag_inconsistente'] = inconsistente

⚠️ ANÁLISIS DE CONSISTENCIA LÓGICA

1. ¿No trabaja pero tiene correspondencia laboral?
   Registros inconsistentes: 0

2. Años de graduación:
anioGraduacion
2024     11
2025    369
Name: count, dtype: int64


---
## 5. Reporte de Filas a Eliminar

In [6]:
# Consolidar flags
df['_eliminar'] = df['_flag_edad_invalida'] | df['_flag_inconsistente']

# También marcar si no tiene asignaturas (para vectorización)
asig_vacia = df.iloc[:, 35].isna() | (df.iloc[:, 35].astype(str).str.strip() == '')

print("=" * 60)
print("📋 REPORTE FINAL DE CALIDAD DE DATOS")
print("=" * 60)

print(f"\nTotal registros: {len(df)}")
print(f"\n🚨 FILAS PROBLEMÁTICAS:")
print(f"   • Edad inválida (<20 o >60): {df['_flag_edad_invalida'].sum()}")
print(f"   • Inconsistencia lógica: {df['_flag_inconsistente'].sum()}")
print(f"   • Sin asignaturas (impacta vectores): {asig_vacia.sum()}")
print(f"\n   📌 TOTAL A REVISAR: {df['_eliminar'].sum()}")

# Tasa de calidad
calidad = 100 * (1 - df['_eliminar'].mean())
print(f"\n✅ TASA DE CALIDAD: {calidad:.1f}%")

📋 REPORTE FINAL DE CALIDAD DE DATOS

Total registros: 380

🚨 FILAS PROBLEMÁTICAS:
   • Edad inválida (<20 o >60): 0
   • Inconsistencia lógica: 0
   • Sin asignaturas (impacta vectores): 1

   📌 TOTAL A REVISAR: 0

✅ TASA DE CALIDAD: 100.0%


---
## 6. Exportar Datos Limpios

In [7]:
# Eliminar filas problemáticas
df_clean = df[~df['_eliminar']].copy()

# Eliminar columnas auxiliares
cols_aux = [c for c in df_clean.columns if c.startswith('_')]
df_clean = df_clean.drop(columns=cols_aux)

# Guardar
df_clean.to_csv(DATA_OUTPUT, index=False)

print("=" * 60)
print("🚀 EXPORTACIÓN COMPLETADA")
print("=" * 60)
print(f"\n📊 Registros originales: {len(df)}")
print(f"📊 Registros limpios: {len(df_clean)}")
print(f"📊 Eliminados: {len(df) - len(df_clean)}")
print(f"\n📁 Archivo guardado: {DATA_OUTPUT}")
print(f"   Tamaño: {DATA_OUTPUT.stat().st_size / 1024:.1f} KB")

🚀 EXPORTACIÓN COMPLETADA

📊 Registros originales: 380
📊 Registros limpios: 380
📊 Eliminados: 0

📁 Archivo guardado: data/processed/clean_survey_data.csv
   Tamaño: 226.2 KB


In [8]:
# Resumen ejecutivo
print("\n" + "=" * 60)
print("📋 RESUMEN EJECUTIVO - DQA")
print("=" * 60)
print(f"""
Dataset: Encuesta Graduados EPN
Registros Analizados: {len(df)}
Tasa de Calidad: {calidad:.1f}%

Problemas Detectados:
• Edades fuera de rango: {df['_flag_edad_invalida'].sum()}
• Inconsistencias lógicas: {df['_flag_inconsistente'].sum()}

Acción Tomada:
• Registros eliminados: {len(df) - len(df_clean)}
• Dataset limpio exportado a: {DATA_OUTPUT}

Próximo paso: 02_Preprocessing_Master.ipynb
""")


📋 RESUMEN EJECUTIVO - DQA

Dataset: Encuesta Graduados EPN
Registros Analizados: 380
Tasa de Calidad: 100.0%

Problemas Detectados:
• Edades fuera de rango: 0
• Inconsistencias lógicas: 0

Acción Tomada:
• Registros eliminados: 0
• Dataset limpio exportado a: data/processed/clean_survey_data.csv

Próximo paso: 02_Preprocessing_Master.ipynb

